# WGS Pipeline

In [1]:
import os
import argparse
import collections
import pandas as pd
import subprocess
from Bio import SeqIO

from itertools import repeat
from multiprocessing import Pool, freeze_support

In [ ]:
mamba create -n wgs pandas fastp unicycler quast bandage kraken2 prokka abricate

In [ ]:
conda install -c bioconda fastp unicycler quast bandage kraken2 prokka diamond

In [ ]:
conda install pandas

## QC

### fastP

https://github.com/OpenGene/fastp

In [6]:
## Fastp pair end tirmming and merging
def RunFastp(R1, R2, prefix, OutDir, threads):
    fastpDir = os.path.join(OutDir, "fastp")
    if os.path.exists(OutDir) == 0:
        os.makedirs(OutDir, 0o777, True)
    if os.path.exists(fastpDir) == 0:
        os.makedirs(fastpDir, 0o777, True)
    cmd = "fastp --in1 " + R1 + " --in2 " + R2 + " --out1 " + os.path.join(fastpDir, prefix + "_R1.fastq") + " --out2 " + os.path.join(fastpDir, prefix + "_R2.fastq") + \
    " --thread " + str(threads) + \
    " --html " + os.path.join(fastpDir, prefix + ".html") + " --json " + os.path.join(fastpDir, prefix + ".json") + " --report_title " + prefix + "-fastq-merge-report"
    subprocess.call(cmd, shell=True)
## Run fastp in parallel
def RunFastpParallel(R1List, R2List, prefixList, OutDir, threads, jobs):
    pool = Pool(processes = jobs)
    pool.starmap(RunFastp, zip(R1List, R2List, prefixList, repeat(thread), repeat(OutDir)))
    pool.close()
    pool.join()
    pool.terminate()

#### test

fastp -i /mnt/d/Lab/WGS-Pipeline/testdata/ERR044595_1M_1.fastq.gz -I /mnt/d/Lab/WGS-Pipeline/testdata/ERR044595_1M_2.fastq.gz -o /mnt/d/Lab/WGS-Pipeline/result/fastp/out.R1.fq.gz -O /mnt/d/Lab/WGS-Pipeline/result/fastp/out.R2.fq.gz

In [10]:
R1 = "/mnt/d/Lab/WGS-Pipeline/testdata/ERR044595_1M_1.fastq.gz"
R2 = "/mnt/d/Lab/WGS-Pipeline/testdata/ERR044595_1M_2.fastq.gz"
prefix = "ERR044595_1M"
OutDir = "/mnt/d/Lab/WGS-Pipeline/result"

In [8]:
RunFastp(R1, R2, prefix, OutDir)

if `-a` add adapter seq

rename `--html` and `--json` output

add `--report_title`

### skewer

https://github.com/relipmoc/skewer

## Assembly

### Unicycler

https://github.com/rrwick/Unicycler

```shell
unicycler -1 $result/trimmomatic/ERR044595_1M_1.paired.fastq -2 $result/trimmomatic/ERR044595_1M_2.paired.fastq -o $result/unicycler
```

In [9]:
def RunUnicycler(R1, R2, prefix, OutDir, threads):
    cmd = "unicycler -1 " + R1 + " -2 " + R2 + " -o " + os.path.join(OutDir, prefix) + " --threads " + str(threads)
    subprocess.call(cmd, shell=True)
def RunUnicyclerParallel(R1List, R2List, prefixList, OutDir, threads, jobs):
    pool = Pool(processes = jobs)
    pool.starmap(RunUnicycler, zip(R1List, R2List, prefixList, repeat(OutDir), repeat(thread)))
    pool.close()
    pool.join()
    pool.terminate()

In [12]:
RunUnicycler(R1, R2, prefix, OutDir, 8)

### Quast

https://github.com/ablab/quast

In [ ]:
* GRIDSS (needed for structural variants detection)
* SILVA 16S rRNA database (needed for reference genome detection in metagenomic datasets)
* BUSCO tools and databases (needed for searching BUSCO genes) -- works in Linux only!

To be able to use those, please run
    quast-download-gridss
    quast-download-silva
    quast-download-busco

In [ ]:
def RunQuast(fasta, R1, R2, prefix, OutDir, threads):
    cmd = "quast.py " + fasta + " -1 " + R1 + " -2 " + R2 + " -o " + os.path.join(OutDir, prefix) + " --threads " + str(threads)
## RunQuastParallel
def RunQuastParallel(fastaList, R1List, R2List, prefixList, OutDir, threads, jobs):
    pool = Pool(processes = jobs)
    pool.starmap(RunQuast, zip(fastaList, R1List, R2List, prefixList, repeat(thread), repeat(OutDir)))
    pool.close()
    pool.join()
    pool.terminate()

### Bandage

https://github.com/rrwick/Bandage

In [ ]:
def RunBandage(InFile, OutFile):
    #InFile: A graph file of any type supported by Bandage
    #OutFile: The image file to be created (must end in '.jpg', '.png' or '.svg')
    cmd = "Bandage image " + InFile + " " + OutFile
    subprocess.call(cmd, shell=True)
#Run Bandage in parallel
def RunBandageParallel(fileList, outFileList, jobs):
    pool = Pool(processes=jobs)
    pool.starmap(RunBandage, zip(fileList, outFileList))
    pool.close()
    pool.join()
    pool.terminate()

### kraken2

https://github.com/DerrickWood/kraken2/wiki

In [ ]:
kraken2 -db /home/junyuchen/3-Resources/Databases/hash.k2d --threads 12 --output --report kraken_out.txt

In [ ]:
kraken2 -db /home/junyuchen/3-Resources/Databases/hash.k2d /home/junyuchen/1-Projects/WGS-Pipeline/result/testout/unicycler/ER064912/assembly.fasta --threads 24

In [ ]:
kraken2 -db /home/junyuchen/3-Resources/Databases/k2_standard_20201202 /home/junyuchen/1-Projects/WGS-Pipeline/result/testout/unicycler/ER064912/assembly.fasta --output test_kraken2.txt --report kraken_out.txt --threads 24

In [ ]:
/home/junyuchen/3-Resources/Databases/k2_standard_20201202

In [ ]:
## Run kraken2
def RunKraken2(fasta, database, OutFile, threads):
    cmd = "kraken2 -db " + database + " " + fasta + " --report " + OutFile + " --threads " + str(threads)
    subprocess.call(cmd, shell=True)
#Run kraken2 in parallel
def RunProkkaParallel(fileList, database, OutFileList, threads, jobs):
    pool = Pool(processes=jobs)
    pool.starmap(RunKraken2, zip(fileList, repeat(database), outFileList, repeat(threads)))
    pool.close()
    pool.join()
    pool.terminate()

#### kraken2 database

https://benlangmead.github.io/aws-indexes/k2

## Annotation

### Prokka

https://github.com/tseemann/prokka

In [ ]:
## Run Prokka
def RunProkka(fasta, prefix, outDir, threads):
    cmd = "prokka --addgenes --prefix " + prefix + " --outdir " + outDir + " --force " + fasta + " --cpus " + str(threads)
    print(cmd)
    subprocess.call(cmd, shell=True)
#Run Prokka in parallel
def RunProkkaParallel(fileList, outFileList, prefixList, threads, jobs):
    pool = Pool(processes=jobs)
    pool.starmap(RunProkka, zip(fileList, prefixList, outFileList, repeat(threads)))
    pool.close()
    pool.join()
    pool.terminate()

### ABRicate

https://github.com/tseemann/abricate

In [ ]:
conda install -c conda-forge -c bioconda -c defaults abricate
abricate --check
abricate --list 

In [ ]:
abricate 

In [ ]:
## Run ABRicate
def RunABRicate(fasta, prefix, OutDir, threads):
    cmd = "abricate " + fasta + " > " + os.path.join(OutDir, prefix + "_abricate.txt") + " --threads " + str(threads)
    subprocess.call(cmd, shell=True)
#Run Prokka in parallel
def RunABRicateaParallel(fileList, prefixList, OutDir, threads, jobs):
    pool = Pool(processes=jobs)
    pool.starmap(RunABRicates, zip(fileList, prefixList, repeat(OutDir), repeat(threads)))
    pool.close()
    pool.join()
    pool.terminate()

### antiSMASH

BGCs

### Pathway

#### kofam_scan

https://github.com/takaram/kofam_scan

## Request

特定功能基因: 
- SCFA（刘红宾)   
- 胆汁酸（周春花）   
- BGC（唐啸宇/司同）      
- VFDB   

Phage-Host

噬菌谱？宿主谱

Phage Seq Mapping Metagenomics data

Ma Lab Metageome sample

which dataset?

## Output

- FastQC
- assembly
- quast
- annotation

In [ ]:
python /home/junyuchen/1-Projects/WGS-Pipeline/Scripts/WGS-Pipeline.py -i /home/junyuchen/1-Projects/WGS-Pipeline/data/testdata.tsv -o /home/junyuchen/1-Projects/WGS-Pipeline/result/testout -t 12 -j 2